# 03-202 : Question 5 Model

Train a model specifically for question 5, and then use it in conjunction with `simple_monkey` to predict the entire dataset and get a classification report.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import logging
from typing import Tuple

import pandas as pd
import mlflow

import tensorflow_addons as tfa
import keras as k
from keras import optimizers
import keras_tuner
import keras_tuner as kt

from competition import data_preparation as dp
from competition import feature_engineering as fe
from competition import model_data as md
from competition import source_data as sd
import competition.models.simple_dense as sd_model
from competition.models.heatmap_covnet import HeatmapCovnetModel

2023-04-02 19:54:29.342470: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Configure Logging

In [3]:
logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S',
        handlers=[
        logging.StreamHandler(sys.stdout)
    ])

logging.info("Started")

2023-04-02 19:54:30 INFO     Started


## Data Collection

In [4]:
# load the source training set
df_source = sd.read_csv('../data/train.csv.gz',
                        compression='gzip',
                        dtype=sd.source_dtype)

(13174211, 20)


,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
0,20090312431273200,0,0,cutscene_click,basic,0,NaN,-413.991394,-159.314682,380.0,494.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,NaN,NaN,NaN,0-4
1,20090312431273200,1,1323,person_click,basic,0,NaN,-413.991394,-159.314682,380.0,494.0,NaN,"Whatcha doing over there, Jo?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
2,20090312431273200,2,831,person_click,basic,0,NaN,-413.991394,-159.314682,380.0,494.0,NaN,Just talking to Teddy.,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4


In [5]:
# load the source training labels
df_source_labels = sd.read_csv('../data/train_labels.csv')

(212022, 2)


,session_id,correct
0,20090312431273200_q1,1
1,20090312433251036_q1,0
2,20090314121766812_q1,1


## Data Preparation & Cleaning

In [6]:
# prepare the main dataset
df_source = dp.prepare_main_dataset(df_source,
                                    elapsed_time_min_clip=0,
                                    elapsed_time_max_clip=3691298)

# remove sessions with problems
problem_sessions = dp.find_problem_sessions(df_source)
df_source = df_source[~df_source['session_id'].isin(problem_sessions)]

In [7]:
# prepare the label dataset
df_source_labels = dp.prepare_label_dataset(df_source_labels)

# remove sessions with problems
df_source_labels = df_source_labels[~df_source_labels['session_id'].isin(problem_sessions)]

## Feature Engineering

Instead of duplicating the feature engineering workflow, we will use the same feature dataset created in notebook `03-123`.

In [8]:
df_features = pd.read_pickle(
    'data/features/03-123.parquet',
    compression='gzip')

with pd.option_context('display.max_columns', None):
    display(df_features.head(3))

,session_id,level_group,elapsed_time_sum,elapsed_time_max,elapsed_time_min,elapsed_time_mean,elapsed_time_mode,count_total_event_name,count_total_name,count_total_fqid,count_total_room_fqid,count_total_text_fqid,count_unique_event_name,count_unique_name,count_unique_fqid,count_unique_room_fqid,count_unique_text_fqid,screen_heatmap_feature
0,20090312431273200,0-4,0.001411,0.052535,0.000000,0.023103,0.000000,0.088782,0.088782,0.064620,0.088782,0.054054,0.75,0.0,0.203390,0.090909,0.225,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
1,20090312431273200,13-22,0.043740,0.344602,0.226677,0.281804,0.301320,0.394721,0.394721,0.402262,0.394721,0.480127,0.75,0.0,0.525424,0.545455,0.675,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
2,20090312431273200,5-12,0.010577,0.135014,0.060002,0.096641,0.060002,0.245951,0.245951,0.276252,0.245951,0.257552,0.75,0.0,0.355932,0.454545,0.400,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."


## Data Selection

We first combine the features with the labels as we will do data selection now based on question number as opposed to to all previous notebooks.

In [9]:
df_combined = df_source_labels.merge(
    right=df_features, 
    on=['session_id', 'level_group'],
    how='left')

print(df_combined.shape)
with pd.option_context('display.max_columns', None):
    display(df_combined.head(6))

(209664, 20)


,session_id,question_num,correct,level_group,elapsed_time_sum,elapsed_time_max,elapsed_time_min,elapsed_time_mean,elapsed_time_mode,count_total_event_name,count_total_name,count_total_fqid,count_total_room_fqid,count_total_text_fqid,count_unique_event_name,count_unique_name,count_unique_fqid,count_unique_room_fqid,count_unique_text_fqid,screen_heatmap_feature
0,20090312431273200,1,1,0-4,0.001411,0.052535,0.0,0.023103,0.0,0.088782,0.088782,0.064620,0.088782,0.054054,0.75,0.000000,0.203390,0.090909,0.225,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
1,20090312433251036,1,0,0-4,0.001352,0.063074,0.0,0.026311,0.0,0.057588,0.057588,0.053312,0.057588,0.050874,1.00,0.333333,0.067797,0.000000,0.075,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
2,20090314121766812,1,1,0-4,0.002928,0.106324,0.0,0.047996,0.0,0.088782,0.088782,0.066236,0.088782,0.044515,1.00,0.333333,0.135593,0.090909,0.175,"[[[0.05882353, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0..."
3,20090314363702160,1,1,0-4,0.001627,0.058690,0.0,0.030143,0.0,0.065987,0.065987,0.050081,0.065987,0.034976,0.50,0.000000,0.050847,0.090909,0.075,"[[[0.05882353, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0..."
4,20090314441803444,1,1,0-4,0.000824,0.047682,0.0,0.020862,0.0,0.019196,0.019196,0.025848,0.019196,0.022258,0.50,0.000000,0.067797,0.090909,0.050,"[[[0.05882353, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0..."
5,20090315081004164,1,0,0-4,0.002515,0.092231,0.0,0.036151,0.0,0.116377,0.116377,0.096931,0.116377,0.063593,0.75,0.000000,0.118644,0.090909,0.150,"[[[0.05882353, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0..."


Create a function that will combine the datasets like we just did above and then return the dataset for the specified question.

In [10]:
def get_question_dataset(features: pd.DataFrame,
                         labels: pd.DataFrame,
                         question_num: int) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Returns a dataset containing only the specified question_num.

    Parameters
    ----------
    features : pd.DataFrame
        The features dataset with prepared and normalized data.
    labels : pd.DataFrame
        The labels dataset containing the target variable.
    question_num : int
        The question number to filter on.

    Returns
    -------
    Tuple[pd.DataFrame, pd.DataFrame]
        The filtered features and labels datasets.
    """
    # combine the features and labels datasets
    df_combined = labels.merge(
        right=features, 
        on=['session_id', 'level_group'],
        how='left')

    # filter the combined dataset on the specified question_num
    df_question = df_combined[df_combined['question_num'] == question_num]

    # convert the "heatmap" column to a list
    screen_heatmap_feature = pd.DataFrame()
    if 'screen_heatmap_feature' in df_question.columns:
        logging.info('Temporarily removing the "screen_heatmap_feature" column')
        screen_heatmap_feature = df_question['screen_heatmap_feature']

    # split the combined dataset into features and labels again
    df_question_features = df_question \
        .drop(columns=['question_num', 'correct', 'screen_heatmap_feature']) \
        .drop_duplicates()
    
    # add the heatmap feature to df_question_features
    if 'screen_heatmap_feature' in df_question.columns:
        logging.info('Adding back the "screen_heatmap_feature" column')
        df_question_features = df_question_features.join(screen_heatmap_feature, how='left')

    df_question_labels = df_question[['session_id', 'question_num', 
                                     'correct', 'level_group']]        
    
    # return the filtered features and labels datasets
    return df_question_features, df_question_labels

# test the function
df_question_features, df_question_labels = get_question_dataset(features=df_features,
                                                                labels=df_source_labels,
                                                                question_num=5)

2023-04-02 19:55:04 INFO     Temporarily removing the "screen_heatmap_feature" column
2023-04-02 19:55:04 INFO     Adding back the "screen_heatmap_feature" column


In [11]:
df_question_features

,session_id,level_group,elapsed_time_sum,elapsed_time_max,elapsed_time_min,elapsed_time_mean,elapsed_time_mode,count_total_event_name,count_total_name,count_total_fqid,count_total_room_fqid,count_total_text_fqid,count_unique_event_name,count_unique_name,count_unique_fqid,count_unique_room_fqid,count_unique_text_fqid,screen_heatmap_feature
46592,20090312431273200,5-12,0.010577,0.135014,0.060002,0.096641,0.060002,0.245951,0.245951,0.276252,0.245951,0.257552,0.75,0.000000,0.355932,0.454545,0.400,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
46593,20090312433251036,5-12,0.021933,0.221287,0.072301,0.150206,0.072301,0.364727,0.364727,0.400646,0.364727,0.238474,1.00,0.333333,0.457627,0.454545,0.350,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
46594,20090314121766812,5-12,0.027818,0.275332,0.118501,0.195489,0.118501,0.352729,0.352729,0.342488,0.352729,0.248013,1.00,0.333333,0.457627,0.454545,0.375,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
46595,20090314363702160,5-12,0.011593,0.149509,0.066941,0.105936,0.066941,0.245951,0.245951,0.279483,0.245951,0.289348,0.75,0.000000,0.372881,0.454545,0.300,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
46596,20090314441803444,5-12,0.009992,0.148915,0.059316,0.104761,0.059316,0.200360,0.200360,0.239095,0.200360,0.219396,0.75,0.000000,0.338983,0.454545,0.275,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58235,22100215342220508,5-12,0.032607,0.424882,0.162989,0.295109,0.162989,0.249550,0.249550,0.297254,0.249550,0.254372,1.00,1.000000,0.457627,0.545455,0.400,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
58236,22100215460321130,5-12,0.027217,0.287996,0.100020,0.192764,0.100020,0.349130,0.349130,0.405493,0.349130,0.340223,1.00,0.333333,0.508475,0.545455,0.425,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
58237,22100217104993650,5-12,0.027509,0.266463,0.119558,0.189858,0.119558,0.361128,0.361128,0.382876,0.361128,0.314785,1.00,0.333333,0.457627,0.545455,0.325,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
58238,22100219442786200,5-12,0.010752,0.151934,0.064350,0.104607,0.064350,0.224355,0.224355,0.235864,0.224355,0.225755,1.00,1.000000,0.389831,0.454545,0.300,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."


Split the data into training and test sets.

In [12]:
random_state = 51

In [13]:
# split the dataset into train, validation and test sets
train, val, test = md.select_sessions(
    y=df_question_labels,
    random_state=random_state,
    test_size=0.60,
    train_size=0.75)

Train: 3495
Validation: 1165
Test: 6988


## Model Training

### Create Datasets

In [14]:
# set the feature list
feature_list = ['elapsed_time_sum', 'elapsed_time_max', 'elapsed_time_min', 'elapsed_time_mean', 'elapsed_time_mode']

# create the simple model dataset
features_dataset = md.get_feature_dataset(
    features=df_question_features,
    y=df_question_labels,
    feature_list=feature_list,
    train=train,
    val=val,
    test=test,
    include_question=True,
    expand_question=False)

2023-04-02 19:55:06 INFO     -- Creating the train dataset
2023-04-02 19:55:06 INFO     Creating the dataset for all level groups


  0%|          | 0/3495 [00:00<?, ?it/s]

2023-04-02 19:55:11 INFO     -- Creating the val dataset
2023-04-02 19:55:11 INFO     Creating the dataset for all level groups


  0%|          | 0/1165 [00:00<?, ?it/s]

2023-04-02 19:55:13 INFO     -- Creating the test dataset
2023-04-02 19:55:13 INFO     Creating the dataset for all level groups


  0%|          | 0/6988 [00:00<?, ?it/s]

In [15]:
# convert the labels for multi-label classification
cat_features_dataset = md.labels_to_categorical(features_dataset)

In [16]:
# get the shape of the question only dataset
input_data = cat_features_dataset['train']['X']
features_dataset_shape = input_data.shape[1]
print('features_dataset_shape:', features_dataset_shape)

# define the output shape
output_data = cat_features_dataset['train']['y']
output_shape = output_data.shape[1]
print('output_shape', output_shape)

features_dataset_shape: 23
output_shape 2


### Flat Features Only

In [17]:
# configure mlflow
mlflow.set_experiment("question-05-simple")
mlflow.keras.autolog()

In [18]:
# create the hyperparameter object
def define_tune_parameters(hp):
    hp.Int('dense_layer_count', min_value=1, max_value=20, step=1)
    hp.Int('dense_units', min_value=512, max_value=1700, step=32)
    hp.Choice('dense_activation', values=['relu', 'tanh', 'LeakyReLU'])
    hp.Float('dense_l1_regularization', min_value=0.0, max_value=0.0005, step=0.00001)
    hp.Float('dense_l2_regularization', min_value=0.0, max_value=0.001, step=0.0001)
    hp.Float('dense_dropout', min_value=0.005, max_value=0.1, step=0.005)
    hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4, 1e-5])

In [ ]:
# find the best model
model = sd_model.tune_model(
    define_tune_parameters=define_tune_parameters,
    dataset=cat_features_dataset,
    max_trials=1000,
    input_shape=features_dataset_shape,
    output_shape=output_shape,
    dense_layer_count='dense_layer_count',
    dense_units='dense_units',
    dense_activation='dense_activation',
    dense_l1_regularization='dense_l1_regularization',
    dense_l2_regularization='dense_l2_regularization',
    dense_dropout='dense_dropout',
    train_epochs=2000,
    train_batch_size=4000,
    train_optimizer=optimizers.Adam,
    train_learning_rate='learning_rate',
    train_loss='categorical_crossentropy',
    train_metrics=[tfa.metrics.F1Score(name='f1_score', num_classes=2, threshold=0.5, average='macro')],
    train_class_weight=None,
    tune_objective='val_f1_score',
    tune_direction='max',
    tuner_type=kt.tuners.RandomSearch)

model.summary()

### Combined Features

In [26]:
# configure mlflow
mlflow.set_experiment("question-05-heatmap")
mlflow.keras.autolog()

In [27]:
# create the heatmap dataset
heatmap_dataset = md.get_feature_dataset(
    features=df_question_features,
    y=df_question_labels,
    feature_list=['screen_heatmap_feature'],
    train=train,
    val=val,
    test=test,
    include_question=False,
    expand_question=False)

2023-04-02 19:57:00 INFO     -- Creating the train dataset
2023-04-02 19:57:00 INFO     Creating the dataset for all level groups


  0%|          | 0/3495 [00:00<?, ?it/s]

2023-04-02 19:57:05 INFO     -- Creating the val dataset
2023-04-02 19:57:05 INFO     Creating the dataset for all level groups


  0%|          | 0/1165 [00:00<?, ?it/s]

2023-04-02 19:57:07 INFO     -- Creating the test dataset
2023-04-02 19:57:07 INFO     Creating the dataset for all level groups


  0%|          | 0/6988 [00:00<?, ?it/s]

In [28]:
# define the flat heatmap input shape
input_data = heatmap_dataset['train']['X']
heatmap_shape = input_data.shape[1], input_data.shape[2], input_data.shape[3]
print('heatmap_shape:', heatmap_shape)

# get the shape of the question only dataset
input_data = cat_features_dataset['train']['X']
features_dataset_shape = input_data.shape[1]
print('features_dataset_shape:', features_dataset_shape)

# define the output shape
output_data = cat_features_dataset['train']['y']
output_shape = output_data.shape[1]
print('output_shape', output_shape)

heatmap_shape: (23, 10, 10)
features_dataset_shape: 23
output_shape 2


In [29]:
# define the tuner parameters
def define_heatmap_tune_parameters(hp):
    # add the simple model parameters
    define_tune_parameters(hp)

    # add the heatmap model parameters
    hp.Int('covnet_block_count', min_value=1, max_value=3, step=1)
    hp.Choice('covnet_activation', values=['relu', 'tanh', 'LeakyReLU'])
    hp.Int('covnet_cov_count', min_value=1, max_value=3, step=1)
    hp.Int('covnet_channels', min_value=32, max_value=64, step=16)
    hp.Choice('covnet_kernel_size', values=['(3, 3)'])
    hp.Choice('covnet_pool_size', values=['(2, 2)'])

In [30]:
# create the model object
heatmap_model = HeatmapCovnetModel(
    input_shape=features_dataset_shape,
    heatmap_shape=heatmap_shape,
    output_shape=output_shape,
    loss='categorical_crossentropy',
    metrics=[tfa.metrics.F1Score(name='f1_score', num_classes=2, threshold=0.5, average='macro')])

In [31]:
mlflow.end_run()

In [32]:
# search for the best model
heatmap_model.tune_model(
    define_tune_parameters=define_heatmap_tune_parameters,
    heatmap_dataset=heatmap_dataset,
    feature_dataset=cat_features_dataset,
    max_trials=100,
    train_epochs=40,
    train_batch_size=4000,
    train_optimizer=optimizers.Adam,
    tuner_type=kt.tuners.RandomSearch,
    tune_objective='val_f1_score',
    tune_direction='max',
    train_class_weight=None)

Trial 11 Complete [00h 00m 27s]
val_f1_score: 0.6656924486160278

Best val_f1_score So Far: 0.6656924486160278
Total elapsed time: 00h 05m 34s

Search: Running Trial #12

Value             |Best Value So Far |Hyperparameter
17                |5                 |dense_layer_count
1184              |1344              |dense_units
tanh              |relu              |dense_activation
0.00011           |0.00019           |dense_l1_regularization
0.0005            |0.0002            |dense_l2_regularization
0.055             |0.065             |dense_dropout
0.001             |1e-05             |learning_rate
2                 |2                 |covnet_block_count
LeakyReLU         |tanh              |covnet_activation
1                 |1                 |covnet_cov_count
32                |64                |covnet_channels
(3, 3)            |(3, 3)            |covnet_kernel_size
(2, 2)            |(2, 2)            |covnet_pool_size

2023-04-02 20:02:52 INFO     Creating a heatmap CNN 

KeyboardInterrupt: 